In [1]:
import numpy as np
import pandas as pd
import random
import copy
import networkx as nx
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score

In [3]:
def game_community_opinion(W, S_RANDOM, X_INI, Lambda = 1.5, BETA = 1, gamma = 0.95, ETA=0.1):
    """
    ################################ 输入参数 ###############################
        Influence matrix W : n*n, 
        initial opinion vector X_INI : n*1, 
        equilibrium initial community membership matrix S_INI : n*c,
        diagonal matrix Lambda_MATRIX : n*n,
        tuning parameters BETA and gamma.
    """
    t = 0
    S_STAR = S_RANDOM
    row_identity = np.ones((n, 1))
    ALL_ONES = np.ones((n, n))
    
    Lambda_MATRIX = Lambda*np.eye(n)
    
    ############### 节点i的邻居节点 ： W_ij + W_ji > 0 and i #####################
    Neighbor_set = []
    Neighbor_MATRIX = W + W.T + np.eye(n)
    for i in range(n):
        Neighbor_set.append(list(np.where(Neighbor_MATRIX[i])[0]))
        
    FINAL_PHI_S = 0
    
    while 1:
        #print('########################### t = %d ######################'%t)
        
        E = X_INI*row_identity.T - row_identity*X_INI.T
        E = np.abs(E)
        D = ETA - E
        B = np.maximum(0, D)
        
        C = np.matmul(Lambda_MATRIX, D) - B
        Y = C*W+C.T*W.T
        
        Z = Y.reshape(n, 1, n)
        
        S_INI = copy.deepcopy(S_STAR)
        S_Y = np.matmul(S_INI.T, Y)
        S_Y_S = np.matmul(S_Y, S_INI)
        PHI_S = 0.5*S_Y_S.trace()
        
        DELTA_PHI = 1e10
        
        ################################  更新社区标签  ################################
        while DELTA_PHI!=0:
            DELTA_PHI = 0
            for i in range(n):
                F_i = S_INI[Neighbor_set[i]]
                S_S = np.matmul(F_i, S_INI.T)
                P_i = np.matmul(S_S, Z[i].T)
                V_i = P_i.argmax()
                S_i_star = F_i[V_i]
                
                S_MINUS_S_S = np.matmul(S_i_star - S_INI[i], S_INI.T)
                DELTA_PHI += np.matmul(S_MINUS_S_S, Z[i].T)
                S_INI[i] = S_i_star
            
            PHI_S += DELTA_PHI[0]
            #print("PHI_S : %d"%PHI_S)
            
        beta_gamma = BETA*pow(gamma, t)
        Xi_t = beta_gamma*ALL_ONES
        relu_D = np.maximum(0, Xi_t - E)
        W_D = W*relu_D
        
        X_S_S = np.matmul(S_INI, S_INI.T)
        Lambda_MODIFY = Lambda*ALL_ONES - 1
        X_S_S_MODIFY = 1 + Lambda_MODIFY*X_S_S
        
        THETA = X_S_S_MODIFY*W_D
        
        GAMMA = np.matmul(THETA, ALL_ONES) + ALL_ONES
        PHI = THETA / GAMMA
        
        PHI_prime = np.matmul(THETA, row_identity) + row_identity
        PHI_prime = row_identity / PHI_prime
        
        X_UPDATE = PHI_prime*X_INI + np.matmul(PHI, X_INI)
        
        S_STAR = S_INI
        
#         print("PHI_S : ",PHI_S)
#         print("FINAL_PHI_S : ",FINAL_PHI_S)
        
        DETA_X = np.abs(X_UPDATE - X_INI)
        if DETA_X.max() < 1e-3:
            break
        
        X_INI = X_UPDATE
        FINAL_PHI_S = PHI_S
        
        t += 1
    
    return S_STAR, X_UPDATE, PHI

In [6]:
"""
##############################  全局变量  #############################
n : 节点数量
c : 初始社区数量
"""
G = nx.read_gml('dolphins.gml')
W = np.array(nx.adjacency_matrix(G).todense())

n = W.shape[0]
ALL_ONES = np.ones((n, n))

G_COMPUTE_Q = nx.from_numpy_array(W)

COMMUNITY_LABELS = np.array([2,6,7,8,10,14,18,20,23,26,27,28,32,33,42,49,55,57,58,61]) - 1
TRUE_COMMUNITY = np.zeros(n)
TRUE_COMMUNITY[list(COMMUNITY_LABELS)] = 1

c = 20

In [8]:
############################# 给定初始社区结构
S = np.zeros((n, c))
random.seed(31)
community_index = [random.randint(0, c-1) for _ in range(n)]
for i in range(n):
    S[i][community_index[i]]=1

o_all_ami = []
o_all_ari = []

for x_seed in range(100):
    np.random.seed(x_seed)
    X = np.random.uniform(0, 1, [n, 1])
    
    FINAL_S, FINAL_X, FINAL_WEIGHT = game_community_opinion(W, S, X, Lambda = 1.2, ETA = 0.4, gamma = 0.9)

    NODES_LABELS = np.where(FINAL_S==1)[1]
    COMMUNITY_LABELS = list(set(NODES_LABELS))
    
    COMMUNITY_PROPAGATION = []
    for i in range(len(COMMUNITY_LABELS)):
        COMMUNITY_LABEL_SET = []
        COMMUNITY_LABEL_ONE = COMMUNITY_LABELS[i]
        for j in range(n):
            if NODES_LABELS[j]==COMMUNITY_LABEL_ONE:
                COMMUNITY_LABEL_SET.append(j)
        COMMUNITY_PROPAGATION.append(COMMUNITY_LABEL_SET)
    
    ari = adjusted_rand_score(TRUE_COMMUNITY, NODES_LABELS)
    ami = adjusted_mutual_info_score(TRUE_COMMUNITY, NODES_LABELS)
    o_all_ari.append(ari)
    o_all_ami.append(ami)

print("ari mean: ", np.array(o_all_ari).mean())
print("ari std: ", np.array(o_all_ari).std())
print("ami mean: ", np.array(o_all_ami).mean())
print("ami std: ", np.array(o_all_ami).std())

ari mean:  0.6180053568051347
ari std:  0.25519494545293814
ami mean:  0.6390029100225979
ami std:  0.216225593116044


In [9]:
####################### 给定初始意见向量
np.random.seed(60)
X = np.random.uniform(0, 1, [n, 1])

c_all_ami = []
c_all_ari = []

for c_seed in range(100):
    S = np.zeros((n, c))
    random.seed(c_seed)
    community_index = [random.randint(0, c-1) for _ in range(n)]
    for i in range(n):
        S[i][community_index[i]]=1
    
    FINAL_S, FINAL_X, FINAL_WEIGHT = game_community_opinion(W, S, X, Lambda = 1.2, ETA = 0.2, gamma = 0.9)

    NODES_LABELS = np.where(FINAL_S==1)[1]
    COMMUNITY_LABELS = list(set(NODES_LABELS))
    
    COMMUNITY_PROPAGATION = []
    for i in range(len(COMMUNITY_LABELS)):
        COMMUNITY_LABEL_SET = []
        COMMUNITY_LABEL_ONE = COMMUNITY_LABELS[i]
        for j in range(n):
            if NODES_LABELS[j]==COMMUNITY_LABEL_ONE:
                COMMUNITY_LABEL_SET.append(j)
        COMMUNITY_PROPAGATION.append(COMMUNITY_LABEL_SET)
    
    ari = adjusted_rand_score(TRUE_COMMUNITY, NODES_LABELS)
    ami = adjusted_mutual_info_score(TRUE_COMMUNITY, NODES_LABELS)
    c_all_ari.append(ari)
    c_all_ami.append(ami)
    
print("ari mean: ", np.array(c_all_ari).mean())
print("ami mean: ", np.array(c_all_ami).mean())

ari mean:  0.958928575339136
ami mean:  0.9668215571322157


In [12]:
####################### LPA社区检测
LPA_ALL_ARI = []
LPA_ALL_AMI = []

for ITER_NUMBER in range(100):
    PREDICT_COMMUNITY = np.zeros(n)
    found_community = list(nx.community.label_propagation_communities(G_COMPUTE_Q))
    for i in range(len(found_community)):
        PREDICT_COMMUNITY[list(found_community[i])] = i
    
    LPA_ARI = adjusted_rand_score(TRUE_COMMUNITY, PREDICT_COMMUNITY)
    LPA_AMI = adjusted_mutual_info_score(TRUE_COMMUNITY, PREDICT_COMMUNITY)
    
    LPA_ALL_ARI.append(LPA_ARI)
    LPA_ALL_AMI.append(LPA_AMI)
    
print('LPA ARI MEANS: ', np.array(LPA_ALL_ARI).mean())
print('LPA AMI MEANS: ', np.array(LPA_ALL_AMI).mean())

LPA ARI MEANS:  0.36136102858828095
LPA AMI MEANS:  0.5050636635504439


In [13]:
####################### kc-clique社区检测
KC_ALL_ARI = []
KC_ALL_AMI = []

for ITER_NUMBER in range(100):
    
    PREDICT_COMMUNITY = np.zeros(n)
    found_community = list(nx.community.k_clique_communities(G_COMPUTE_Q, 3))
    for i in range(len(found_community)):
        PREDICT_COMMUNITY[list(found_community[i])] = i

    KC_ARI = adjusted_rand_score(TRUE_COMMUNITY, PREDICT_COMMUNITY)
    KC_AMI = adjusted_mutual_info_score(TRUE_COMMUNITY, PREDICT_COMMUNITY)
    
    KC_ALL_ARI.append(KC_ARI)
    KC_ALL_AMI.append(KC_AMI)
    
print('k-clique ARI MEANS: ', np.array(KC_ALL_ARI).mean())
print('k-clique AMI MEANS: ', np.array(KC_ALL_AMI).mean())    

k-clique ARI MEANS:  0.31777330356622696
k-clique AMI MEANS:  0.4261074678214524


In [14]:
ALL_AMI = pd.DataFrame(np.array(o_all_ami), columns = ['GCAOFP-O'])
ALL_AMI['GCAOFP-C'] = c_all_ami
ALL_AMI['LPA'] = LPA_ALL_AMI
ALL_AMI['KC'] = KC_ALL_AMI

ALL_ARI = pd.DataFrame(np.array(o_all_ari), columns = ['GCAOFP-O'])
ALL_ARI['GCAOFP-C'] = c_all_ari
ALL_ARI['LPA'] = LPA_ALL_ARI
ALL_ARI['KC'] = KC_ALL_ARI

In [15]:
ALL_AMI.to_excel('DOLPHINS_community_AMI.xlsx', index=False)
ALL_ARI.to_excel('DOLPHINS_community_ARI.xlsx', index=False)

In [16]:
ALL_AMI

,GCAOFP-O,GCAOFP-C,LPA,KC
0,0.693209,1.000000,0.505064,0.426107
1,0.643354,1.000000,0.505064,0.426107
2,0.426910,0.635728,0.505064,0.426107
3,0.579776,1.000000,0.505064,0.426107
4,0.567574,1.000000,0.505064,0.426107
...,...,...,...,...
95,0.252369,1.000000,0.505064,0.426107
96,0.444915,0.635728,0.505064,0.426107
97,0.213207,1.000000,0.505064,0.426107
98,0.465609,1.000000,0.505064,0.426107


In [17]:
ALL_ARI

,GCAOFP-O,GCAOFP-C,LPA,KC
0,0.760667,1.000000,0.361361,0.317773
1,0.474725,1.000000,0.361361,0.317773
2,0.296925,0.452008,0.361361,0.317773
3,0.447642,1.000000,0.361361,0.317773
4,0.412728,1.000000,0.361361,0.317773
...,...,...,...,...
95,0.273526,1.000000,0.361361,0.317773
96,0.318893,0.452008,0.361361,0.317773
97,0.006092,1.000000,0.361361,0.317773
98,0.453728,1.000000,0.361361,0.317773
